In [1]:
import os
# import torch
import numpy as np
import pandas as pd
# from torch.utils.data import Dataset
import fitz
from tqdm import tqdm
import re

In [2]:
full_data = pd.read_csv('Дата-сет для задачи №1\\Дата-сет_Задача 1.csv', sep=';', encoding="windows-1251")

In [3]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100425 entries, 0 to 100424
Data columns (total 5 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Номер_в_госреестре              100416 non-null  object 
 1   Наименование_СИ                 99851 non-null   object 
 2   Единица_измерения_СИ            0 non-null       float64
 3   Погрешность_СИ                  0 non-null       float64
 4   Наименование_файла_с_описанием  51694 non-null   object 
dtypes: float64(2), object(3)
memory usage: 3.8+ MB


In [4]:
#

have_file_df = full_data[(full_data['Наименование_файла_с_описанием'].notna())]
print(have_file_df.shape)
test_files = os.listdir('Дата-сет для задачи №1\\Разметка')
print(len(test_files))
answer_df = full_data[full_data['Наименование_файла_с_описанием'].isin(test_files)]
print(answer_df.shape)
answer_df = answer_df.drop_duplicates(['Наименование_файла_с_описанием'])
print(answer_df.shape)

In [11]:
file_names=[]
for i in range(answer_df.shape[0]):
    name = answer_df['Наименование_файла_с_описанием'].iloc[i]
    if os.path.exists(f'Дата-сет для задачи №1\\Разметка\\{name}'):
        file_names.append(f'Дата-сет для задачи №1\\Разметка\\{name}')

In [12]:
file_small_names = [name.split('\\')[-1] for name in file_names]
with open('file_small_names.txt', 'w') as f:
    for item in file_small_names:
        f.write("%s\n" % item)


### PDF READER

In [72]:
new_data_dict = {}
data_dict = {}
for file_name in tqdm(file_names):
    # list to store the co-ordinates of all highlights
    highlights = []
    # List to store all the highlighted texts
    highlight_text = []
    # read file
    doc = fitz.open(file_name)
    page_count = doc.page_count
    for n in range(page_count):
        # load page
        page = doc.load_page(n)
        # annot generator
        annot = page.firstAnnot
        # loop till we have highlight annotation in the page
        while annot:
            if annot.type[0] == 8:
                all_coordinates = annot.vertices
                if len(all_coordinates) == 4:
                    highlight_coord = fitz.Quad(all_coordinates).rect
                    highlights.append(highlight_coord)
                else:
                    all_coordinates = [all_coordinates[x:x+4] for x in range(0, len(all_coordinates), 4)]
                    for i in range(0,len(all_coordinates)):
                        coord = fitz.Quad(all_coordinates[i]).rect
                        highlights.append(coord)
            annot = annot.next

        # all words from page
        all_words = page.get_text_words()
        
        for h in highlights:
            sentence = [w[4] for w in all_words if fitz.Rect(h).intersects(w[0:4])]
            highlight_text.append(" ".join(sentence))    

        data_dict[file_name] = highlight_text
        new_data_dict[file_name.split('\\')[-1]] = highlight_text

100%|██████████| 886/886 [03:36<00:00,  4.08it/s]


In [71]:
# Мой датафрейм
df = {i: list(item) for i, item in enumerate(data_dict.items())}
df = pd.DataFrame.from_dict(df, orient='index', columns=['file_name', 'text'])


In [73]:
# Общий датафрейм
df = {i: list(item) for i, item in enumerate(new_data_dict.items())}
df = pd.DataFrame.from_dict(df, orient='index', columns=['file_name', 'text'])
df.to_csv('train_df.csv')

In [74]:
df

,file_name,text
0,2011-46695-11.pdf,[Системы весоизмерительные многоплатформенные ...
1,2011-46750-11.pdf,[Система автоматизированная информационно-изме...
2,2019-58208-14-1.pdf,"[Весы автомобильные ВАС, В качестве весоизмери..."
3,2012-49648-12.pdf,[Датчики силоизмерительные тензорезисторные S3...
4,2019-57469-14.pdf,"[Счетчики тепловой энергии ультразвуковые CF, ..."
...,...,...
881,2020-79291-20.pdf,[Твердомеры стационарные Бринелля NOVOTEST TС-...
882,2005-30815-05.pdf,"[Трансформатор напряжения эталонный, мобильный..."
883,2021-44940-10.pdf,"[cnyrumconurx cucreer GSS8000, I hlllT HT OP b..."
884,2021-43863-10.pdf,[AHam133TOpbl «I>a30Bbix urysron\n aapuaunn An...


### Подготовка датасета для обучения классификатора